<a href="https://colab.research.google.com/github/helldoodle-dev/voice-ai/blob/main/Changer_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Real-Time Voice Changer
##### 😉
---
Colab by helldoodle based on the [voice-ai](https://github.com/helldoodle)

In [ ]:
# @title Копирование репозитория и установка библиотек
import os
import sys
import subprocess
from tqdm import tqdm
from google.colab import drive
from IPython.display import clear_output


# @markdown ---
#
# @markdown Использование Google Диска может немного сократить время загрузки, и ваши модели будут сохранены, поэтому вам не придется повторно загружать их каждый раз, когда вы используете.

use_drive = True # @param {type:"boolean"}

if use_drive is True:
    if not os.path.exists("/content/drive"):
        drive.mount('/content/drive')

    os.chdir('/content/drive/MyDrive')

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

def update_file(file_path, search_str, replace_str):
    with open(file_path, 'r') as f:
        content = f.read()
    content = content.replace(search_str, replace_str)
    with open(file_path, 'w') as f:
        f.write(content)

pathloc = "/content/drive/MyDrive/voice-ai"

!git clone https://github.com/helldoodle-dev/voice-ai.git --quiet
os.chdir(pathloc)

!apt-get -y install libportaudio2 -qq

update_file('./client/demo/dist/assets/gui_settings/version.txt', '-.-.-.-', 'Colab.Mod')
#update_file('./client/demo/dist/index.js', 'inputChunkNum:48', 'inputChunkNum:192')


with open(f'{pathloc}/requirements.txt', 'r') as file:
    packages = file.read().splitlines()

for package in tqdm(packages):
    install(package)

clear_output()
print(f"> Все библиотеки успешно установлены!")

In [ ]:
#@title Запуск ngrok и voice-ai сервера
#@markdown ## <- Запуск, но перед этим:

#@markdown Введите **Ваш** токен [Ngrok](https://dashboard.ngrok.com/get-started/your-authtoken):
token = "1sZBPtLE5JrPOMMkovSyEFgcRxQ_6mN4xN1sFCNXh8BGin6vt" #@param {type:"string"}

#@markdown ...

#@markdown Выберите ближайший сервер к вам:
region = "us - United States (Ohio)" # @param ["ap - Asia/Pacific (Singapore)", "au - Australia (Sydney)","eu - Europe (Frankfurt)", "in - India (Mumbai)","jp - Japan (Tokyo)","sa - South America (Sao Paulo)", "us - United States (Ohio)"]

#@markdown ---

#@markdown *Очищать консоль?
clear_console = True # @param {type:"boolean"}

#@markdown *Проиграть звук?
play_notification = True # @param {type:"boolean"}


os.chdir(pathloc)

from pyngrok import conf, ngrok
MyConfig = conf.PyngrokConfig()
MyConfig.auth_token = token
MyConfig.region = region[0:2]
conf.set_default(MyConfig);

import subprocess, threading, time, socket, urllib.request
PORT = 8000

from pyngrok import ngrok
ngrokConnection = ngrok.connect(PORT)
public_url = ngrokConnection.public_url

from IPython.display import clear_output
from IPython.display import Audio, display
def play_notification_sound():
    display(Audio(url='https://aternos.org/panel/sound/start.mp3', autoplay=True))

def wait_for_server():
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', PORT))
        if result == 0:
            break
        sock.close()

    if clear_console:
        clear_output()
    print("--------- SERVER READY! ---------")
    print("Your server is available at:")
    print(public_url)
    print("---------------------------------")
    if play_notification is True:
        play_notification_sound()

threading.Thread(target=wait_for_server, daemon=True).start()

!python3 MMVCServerSIO.py \
  -p {PORT} \
  --https False \
  --content_vec_500 pretrain/checkpoint_best_legacy_500.pt \
  --content_vec_500_onnx pretrain/content_vec_500.onnx \
  --content_vec_500_onnx_on true \
  --hubert_base pretrain/hubert_base.pt \
  --hubert_base_jp pretrain/rinna_hubert_base_jp.pt \
  --hubert_soft pretrain/hubert/hubert-soft-0d54a1f4.pt \
  --nsf_hifigan pretrain/nsf_hifigan/model \
  --crepe_onnx_full pretrain/crepe_onnx_full.onnx \
  --crepe_onnx_tiny pretrain/crepe_onnx_tiny.onnx \
  --rmvpe pretrain/rmvpe.pt \
  --model_dir model_dir \
  --samples samples.json

ngrok.disconnect(ngrokConnection.public_url)